In [14]:
import os
import sys
import pyodbc
import numpy as np
import calendar
import operator
import pandas as pd
import time
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from datetime import timedelta


import clv_scoring_functions
##(make sure our CLV scoring functions script is accessible)
import glob
from dateutil.relativedelta import relativedelta
import sklearn as sklearn
from sklearn import metrics
from scipy.optimize import minimize, rosen, rosen_der


In [15]:
from dateutil import rrule

def weeks_between(start_date, end_date):
    weeks = rrule.rrule(rrule.WEEKLY, dtstart=start_date, until=end_date)
    return weeks.count()

In [16]:

def add_years(d, years):
    """Return a date that's `years` years after the date (or datetime)
    object `d`. Return the same calendar date (month and day) in the
    destination year, if it exists, otherwise use the following day
    (thus changing February 29 to March 1).

    """
    try:
        return d.replace(year = d.year + years)
    except ValueError:
        return d + (date(d.year + years, 1, 1) - date(d.year, 1, 1))

In [17]:
#Format our date inputs  
if str(type(end_date)) != "<class 'pandas._libs.tslibs.timestamps.Timestamp'>":
    print(type(start_date))
    i = str(start_date)
    try:
        dt_start = datetime.datetime.strptime(i, '%Y/%m/%d')
    except ValueError:
        print("Incorrect start format")

    i = str(end_date)    
    try:
        dt_end = datetime.datetime.strptime(i, '%Y/%m/%d')
    except ValueError:
        print("Incorrect start format")

#Formatting the dates for SQL 
        dt_start = str(dt_start) + str('.000')
        dt_end = str(dt_end) + str('.000')
else: 


SyntaxError: unexpected EOF while parsing (<ipython-input-17-048624321137>, line 19)

In [18]:
def SQL_Query(start_date, end_date, UID, password ):

    dt_start= start_date
    dt_end = end_date    


#Establish our connection to the server 
    conn_string = 'DRIVER=/usr/local/lib/libtdsodbc.so;SERVER=wnj-datasizesql;PORT=1433;DATABASE=Data Analytics;UID={};PWD={}'.format(UID,password)
    conn = pyodbc.connect(conn_string)

#Run our sql query

    query = ("""
    SELECT b.customer_id,
    round(b.recency + b.frequency/10 + b.AmountPerWeek/100, 2)AS RFM,
    b.recency, b.frequency, b.T, 
    b.order_count,
    b.total_amount,
    b.FirstOrderDate, b.LastOrderDate
    FROM
      (

        SELECT
          a.customer_id,
          a.order_count,
          a.recency,
          a.frequency,
          a.T,
          round(a.total_amount /isnull(NULLIF(a.frequency, 0), 1), 2)AS AmountPerWeek,
          a.total_amount,
          a.FirstOrderDate,
          a.LastOrderDate
        FROM
          (
            SELECT
              cbs.customer_identifier AS customer_id,
              count(DISTINCT cast(DATEPART(WK, td.transaction_date)AS VARCHAR)+cast(year(td.transaction_date)AS VARCHAR))- 1
              AS frequency,

              datediff(WEEK,MIN(td.transaction_date),MAX(
                  td.transaction_date))AS recency,
              datediff(WEEK,MIN(td.transaction_date),
                       getdate())    AS T,
              count(distinct(td.av_transaction_id))as order_count,
              MIN(
                  td.transaction_date)  AS FirstOrderDate,
              MAX(
                  td.transaction_date)  AS LastOrderDate,
              round(SUM((td.gross_line_amount - td.pos_discount_amount)),2) AS total_amount
            FROM transaction_detail
              as td
              INNER JOIN customer_basket_scv_2017 AS cbs  ON
                td.av_transaction_id = cbs.transaction_id
            WHERE
              td.transaction_date >='{}'
            AND  
                td.transaction_date < '{}'
              AND cbs.customer_identifier IS NOT NULL
              AND ( td.sku_id is NOT NULL OR td.line_object_description IN ('OMS Sales Tax',
'S6 Special Order T2 Sales Tax',
'S6 T2 Sales Tax',
'-Order T2 Sales Tax',
'OMS T2 Sales Tax',
'S6 Sales Tax',
'-Order Sales Tax',
'S6 Special Order Sales Tax'))
            GROUP BY cbs.customer_identifier
          ) a
      ) b
    ORDER BY RFM DESC, AmountPerWeek DESC, total_amount DESC
""".format(dt_start, dt_end))
    

    
    start_time = time.time()

    trFrame = pd.read_sql_query(query, conn)

    conn.close()

    end_time = time.time()

    run_time = (end_time-start_time)/60

    print(run_time)
    return(trFrame)

In [ ]:
#[['2017/01/01', '2017/12/31'] 
date_list = [ ['2018/01/01', '2018/03/30'], ['2018/01/01', '2018/02/28'],
            ['2018/01/01', '2018/06/30'], ['2018/01/01', '2018/01/31']]

In [19]:
date_list = [['2018/01/01', '2019/02/28']]

In [20]:
totalTransDF = pd.DataFrame()

for datePairs in date_list: 
    start_date = datePairs[0]
    end_date = datePairs[1]
    print(start_date, end_date)
    tempDF = SQL_Query(start_date, end_date, 'Dhananjay.Kumar','password1!')
    
    start_date_str = str(start_date)
    start_date_str = start_date_str.replace("/", "_")
    end_date_str = str(end_date)
    end_date_str = end_date_str.replace("/", "_")
    tempDF['period']= start_date_str + '_'+end_date_str
    tempDF.to_csv(r'/Users/sanjaygopinath/Documents/Data/Churn_Analysis/'+start_date_str+'_'+ end_date_str+'.csv')
    #dfList = [totalTransDF, tempDF]
    #totalTransDF = pd.concat(dfList)

2018/01/01 2019/02/28
39.60963578621546


In [22]:
tempDF.LastOrderDate.max()

Timestamp('2019-02-02 00:00:00')

In [ ]:
# Create a list of dates that iterate from the first day of the year weekly
dates = list()
month = 11
year = 2017
day = 17
first_day = datetime.date(year, month, day)
while year <=2018:

    num_days = calendar.monthrange(year, month)
    print(num_days)
    #first_day = datetime.date(year, month, day)
    
    #last_day = datetime.date(year, month, num_days[1])
    #last_day = first_day + timedelta(days=num_days[1])
    last_day = first_day + timedelta(days=1)
    dates.append([first_day, last_day]) 
    first_day = first_day+timedelta(days=1)
    month = last_day.month

In [ ]:
dates = list()
first_day = datetime.date(2016, 11, 1)
last_day = datetime.date(2017, 11, 23)

dates.append([first_day, last_day]) 

    

In [ ]:
dates = dates[:8]

In [ ]:
dates

In [ ]:
dates[0][1] = datetime.date(2018, 9, 1)

In [ ]:
for datePairs in dates: 
    print(datePairs)
    start_date = datePairs
    end_date = datePairs
    

In [ ]:
start_date

In [ ]:
totalTransDF = pd.DataFrame()

for datePairs in dates: 
    start_date = datePairs[0]
    end_date = datePairs[1]
    print(start_date, end_date)
    tempDF = SQL_Query(start_date, end_date, 'Dhananjay.Kumar','password1!')
    tempDF['period']= start_date
    start_date_str = str(start_date)
    tempDF.to_csv(r'/Users/sanjaygopinath/Documents/Data/Responsys/summary'+start_date_str+'.csv')
    dfList = [totalTransDF, tempDF]
    totalTransDF = pd.concat(dfList)

In [ ]:
#save our daily data so we can then look at CLV for those who shop over the week including black friday
#dailyTransDF = totalTransDF

In [ ]:
totalTransDF['period'].value_counts()

In [ ]:
totalTransDF.groupby(totalTransDF['period']).agg(['mean', 'var'])

In [ ]:
tempDF['total_amount'].sum()

In [ ]:
tempDF.to_csv(r'sept17-sept18.csv')

In [ ]:
del dfList

# Declare our functions